# Backpropagation with zero initialization

* W/b 가 전부 0이면 학습이 안 된다.
* 근데 아예 안 되는건 아닌 것 같다.
* 뭔소리여?
* 한 레이어만 0이라면 학습이 될까?
* 이론적으로는 안 되어야 하지 않나?
* 테스트 해 보자.

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [9]:
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


## Build nets

In [34]:
epoch_n = 3
batch_size = 100
N = mnist.train.num_examples
n_iter = N // batch_size

In [54]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# 이거 W/b 직접 구현하는게 더 나을 것 같은데
h1 = tf.layers.dense(X, 256, activation=tf.nn.relu, kernel_initializer=None)
h2 = tf.layers.dense(h1, 256, activation=tf.nn.relu, kernel_initializer=None)
logits = tf.layers.dense(h2, 10, kernel_initializer=tf.zeros_initializer, bias_initializer=tf.zeros_initializer)

# conv
# net = tf.reshape(X, [-1, 28, 28, 1])
# net = tf.layers.conv2d(net, 32, kernel_size=[3,3], strides=1, kernel_initializer=None, activation=tf.nn.relu)
# net = tf.layers.max_pooling2d(net, pool_size=[2,2], strides=2)
# net = tf.layers.conv2d(net, 64, kernel_size=[3,3], strides=1, kernel_initializer=tf.zeros_initializer, activation=tf.nn.relu)
# net = tf.layers.max_pooling2d(net, pool_size=[2,2], strides=2)
# net = tf.contrib.layers.flatten(net)
# logits = tf.layers.dense(net, 10, kernel_initializer=None)

prob = tf.nn.softmax(logits)

# mean for mini-batch
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, axis=1), tf.argmax(y, axis=1)), tf.float32))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))

train_op = tf.train.AdamOptimizer().minimize(loss)

In [55]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epoch_n):
    avg_acc = 0
    avg_loss = 0
    for _ in range(n_iter):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        _, cur_train_acc, cur_train_loss = sess.run([train_op, accuracy, loss], {X: batch_x, y: batch_y})
        
        avg_acc += cur_train_acc
        avg_loss += cur_train_loss
    
    avg_acc /= n_iter
    avg_loss /= n_iter
    
#     test_avg_acc = 0
#     test_avg_loss = 0
#     for _ in range(mnist.test.num_examples // 1000):
#         batch_x, batch_y = mnist.test.next_batch(1000)
#         cur_acc, cur_loss = sess.run([accuracy, loss], {X: batch_x, y: batch_y})
        
#         avg_acc += cur_train_acc
#         avg_loss += cur_train_loss
    
    test_acc, test_loss = sess.run([accuracy, loss], {X: mnist.test.images, y: mnist.test.labels})
    
    print("[{}/{}] (train) acc: {:.2%}, loss: {:.3f} | (test) acc: {:.2%}, loss: {:.3f}".
          format(epoch+1, epoch_n, avg_acc, avg_loss, test_acc, test_loss))
    

[1/3] (train) acc: 90.09%, loss: 0.346 | (test) acc: 95.89%, loss: 0.137
[2/3] (train) acc: 96.36%, loss: 0.122 | (test) acc: 96.46%, loss: 0.112
[3/3] (train) acc: 97.55%, loss: 0.081 | (test) acc: 97.53%, loss: 0.079


## Test results

* conv/fc 상관없이 다 안 된다.
* 다만, 마지막 레이어는 상관없다.
* 마지막 레이어가 되는 이유는 그냥 백프로파게이션 식을 계산해보면 알 수 있음.
* 마지막 레이어의 gradient 에는 W 가 안 들어가서, W 가 0 이라도 학습이 된다. 다른 레이어는 W 가 곱해지기 때문에 그라디언트가 0이 됨.